In [51]:
# es.indices.delete(index='care')

ObjectApiResponse({'acknowledged': True})

In [1]:
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info().body

{'name': 'a976664ec282',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'ZUf-fEbtSsGvg5jQ7aN4Gw',
 'version': {'number': '8.7.0',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '09520b59b6bc1057340b55750186466ea715e30e',
  'build_date': '2023-03-27T16:31:09.816451435Z',
  'build_snapshot': False,
  'lucene_version': '9.5.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [2]:
care_data = [
    {
    'name': 'Wellness / Sick Visit Codes',
    'short_description': 'Provides wellness and sick visit codes traditionally.',
    'long_description': 'Provides tables of mostly CPT codes related to wellness and sick visits. There are a couple of ICD-10-CM and even a modifier. Each code(s) has a description attached.',
    'type': 'Dashboard & Reports',
    'source': 'Github',
    'therapeutic_area': 'Oncology',
    'dataset':'Claims - Optum',
    'created_date': '2021-12-18'
    },
    {
    'name': 'Lynparza Ovarian Cancer Market Definition',
    'short_description': 'Codes to identify OC patients  and their attributes from claims data.',
    'long_description': 'Contains list of diagnosis codes for identifying OC and metastatic patients, CPT Codes for BRCA/HRD Testing.',
    'type': 'Tools',
    'source': 'Figma',
    'therapeutic_area': 'Vaccines',
    'dataset':'HCP Data',
    'created_date': '2022-01-29'
    },
    {
    'name': 'RCC RWD rulebook',
    'short_description': 'Secondary data in RCC',
    'long_description': 'Provides needed ICD, CPT, J, NCD code related to RCC in secondary data as well as rules on regimen hierarchy.',
    'type': 'Market Definitions',
    'source': 'Confluence',
    'therapeutic_area': 'Chronic Care',
    'dataset':'Claims - IQVIA',
    'created_date': '2022-07-02'
    },
    {
    'name': 'cSCC Model',
    'short_description': 'Python Package for cSCC Model',
    'long_description': 'Provides end users with a financial forecast and reports based on specified inputs.',
    'type': 'Tools',
    'source': 'Sharepoint',
    'therapeutic_area': 'Vaccines',
    'dataset':'Account Data',
    'created_date': '2022-10-25'
    },
    {
    'name': 'IQVIA RCC Cancer Patient Cohort',
    'short_description': 'Get cohort patient for RCC Cancer Primary Tumor.',
    'long_description': 'Get cohort patient for RCC Cancer using PTL logic from IQVIA datasource.',
    'type': 'ML Models',
    'source': 'Figma',
    'therapeutic_area': 'PAN TA',
    'dataset':'Source Agnostic',
    'created_date': '2022-12-22'
    }
    
]
for data in care_data: es.index(index='care', document=data)

### Single Option Search (type/source/therapeutic_area/dataset)

In [54]:
api_response = es.search(index="care", query={"match": {'type':'Tools'}})
for hit in api_response['hits']['hits']:
    print(hit['_score'], end=" - ")
    print(hit['_source'])

1.0341108 - {'name': 'Lynparza Ovarian Cancer Market Definition', 'short_description': 'Codes to identify OC patients  and their attributes from claims data.', 'long_description': 'Contains list of diagnosis codes for identifying OC and metastatic patients, CPT Codes for BRCA/HRD Testing.', 'type': 'Tools', 'source': 'Figma', 'therapeutic_area': 'Vaccines', 'dataset': 'HCP Data', 'created_date': '2022-01-29'}
1.0341108 - {'name': 'cSCC Model', 'short_description': 'Python Package for cSCC Model', 'long_description': 'Provides end users with a financial forecast and reports based on specified inputs.', 'type': 'Tools', 'source': 'Sharepoint', 'therapeutic_area': 'Vaccines', 'dataset': 'Account Data', 'created_date': '2022-10-25'}


In [55]:
api_response = es.search(
    index="care", query={"term": {"therapeutic_area.keyword": "PAN TA"}}
)
for hit in api_response['hits']['hits']:
    print(hit['_score'], end=" - ")
    print(hit['_source'])

1.3862942 - {'name': 'IQVIA RCC Cancer Patient Cohort', 'short_description': 'Get cohort patient for RCC Cancer Primary Tumor.', 'long_description': 'Get cohort patient for RCC Cancer using PTL logic from IQVIA datasource.', 'type': 'ML Models', 'source': 'Figma', 'therapeutic_area': 'PAN TA', 'dataset': 'Source Agnostic', 'created_date': '2022-12-22'}


In [56]:
api_response = es.search(
    index="care",
    query={
        "bool": {
            "must": [
                # {"term": {"source.keyword": "Sharepoint"}},
                {"term": {"dataset.keyword": "Account Data"}},
            ]
        }
    },
)
for hit in api_response['hits']['hits']:
    print(hit['_score'], end=" - ")
    print(hit['_source'])

1.3862942 - {'name': 'cSCC Model', 'short_description': 'Python Package for cSCC Model', 'long_description': 'Provides end users with a financial forecast and reports based on specified inputs.', 'type': 'Tools', 'source': 'Sharepoint', 'therapeutic_area': 'Vaccines', 'dataset': 'Account Data', 'created_date': '2022-10-25'}


In [57]:
api_response = es.search(
    index="care", query={"fuzzy": {"dataset": {"value": "optmu"}}}
)
for hit in api_response['hits']['hits']:
    print(hit['_score'], end=" - ")
    print(hit['_source'])

1.1090355 - {'name': 'Wellness / Sick Visit Codes', 'short_description': 'Provides wellness and sick visit codes traditionally.', 'long_description': 'Provides tables of mostly CPT codes related to wellness and sick visits. There are a couple of ICD-10-CM and even a modifier. Each code(s) has a description attached.', 'type': 'Dashboard & Reports', 'source': 'Github', 'therapeutic_area': 'Oncology', 'dataset': 'Claims - Optum', 'created_date': '2021-12-18'}


### Multi Options Queries

In [58]:
# Taking input from the user
user_input = input("Please enter something: ")

api_response = es.search(
    index="care", query= {"multi_match": {"query": user_input,"fields": ["name", "short_description", "long_description"]}}
)
for hit in api_response['hits']['hits']:
    print(hit['_score'], end=" - ")    
    print(hit['_source'])

1.6217027 - {'name': 'IQVIA RCC Cancer Patient Cohort', 'short_description': 'Get cohort patient for RCC Cancer Primary Tumor.', 'long_description': 'Get cohort patient for RCC Cancer using PTL logic from IQVIA datasource.', 'type': 'ML Models', 'source': 'Figma', 'therapeutic_area': 'PAN TA', 'dataset': 'Source Agnostic', 'created_date': '2022-12-22'}


### optimized results

In [4]:
# Taking input from the user
user_input = input("Please enter something: ")

# Function to perform multi-word fuzzy and regex queries
def perform_queries():
    # Regex query
    combined_query = {
    "query": {
        "bool": {
            "must": [
                {
                    "multi_match": {
                        "query": user_input,
                        "fields": ["name", "short_description", "long_description"],
                        "fuzziness": "AUTO"
                    }
                }
            ],
            "should": [
                {"regexp": {"name": f".*{user_input}.*"}},
                {"regexp": {"short_description": f".*{user_input}.*"}},
                {"regexp": {"long_description": f".*{user_input}.*"}}
            ],
            "must": [{"match": {'type':'ml models'}}],
            "must": [{"match": {'source':'figma'}}],
            "must": [{"match": {'therapeutic_area':'pan ta'}}],
            "must": [{"match": {'dataset':'source agnostic'}}],
            "must": [{"range": {"created_date": {"gte": "2022-10-30"}}}],
        
        }
    }
}

    # Execute the queries
    regex_results = es.search(index='care', body=combined_query)

    return regex_results

# Perform queries
regex_results = perform_queries()

# Print query results
print("\nRegex Query Results:")
if len(regex_results["hits"]["hits"]) == 0:
        print("Your search returned no results.")
else:
    for hit in regex_results['hits']['hits']:
        print(hit['_score'], end=" - ")    
        print(hit['_source']['name'])

C:\Users\ASUS\AppData\Local\Temp\ipykernel_1876\1818658249.py:36: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  regex_results = es.search(index='care', body=combined_query)



Regex Query Results:
1.0 - IQVIA RCC Cancer Patient Cohort
